Initiales setup für GPU Verwendung

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

--2022-06-30 07:42:27--  https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2306981744 (2.1G) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb’

cuda-repo-ubuntu200 100%[===================>]   2.15G  53.0MB/s    in 42s     

2022-06-30 07:43:09 (52.6 MB/s) - ‘cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb’ saved [2306981744/2306981744]

Selecting previously unselected package cuda-repo-ubuntu2004-11-0-local.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu2004-11-0-local_11.0.3-450.51.06-1_amd64.deb ...
Unpacking cuda-repo-ubuntu2004-

Installieren der Dependencies

In [ ]:
!pip install spacy==3.3.1 
!pip install spacy-transformers
!pip install -U spacy[transformers,cuda113]
#for transformers v4.0.0+ and models that require SentencePiece (e.g., ALBERT, CamemBERT, XLNet, Marian, and T5)
!pip install transformers[sentencepiece]
!pip install spacy-sentence-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 325 kB/s 
     |████████████████████████████████| 4.2 MB 10.3 MB/s 
     |████████████████████████████████| 1.1 MB 54.3 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 101 kB 14.2 MB/s 
     |████████████████████████████████| 6.6 MB 54.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74.5 MB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.3 

In [ ]:
!pip install shap

Laden der Spacy Modelle

In [ ]:
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 22 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
!python -m spacy download de_core_news_lg
!python -m spacy download de_dep_news_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 567.8 MB 26 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.8 MB 31 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_dep_news_trf')


In [ ]:
# Importing libraries

import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers

# Storing docs in binary format
from spacy.tokens import DocBin

In [ ]:
# Reading the dataset
df = pd.read_csv("all-data.csv", encoding='latin-1',  names=["Sentiment", "Text"])
df.head()

,Sentiment,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [ ]:
#Splitting the dataset into train and test
train = df.sample(frac = 0.8, random_state = 25)
test = df.drop(train.index)

train['Text']

1171    Altia 's operating profit jumped to EUR 47 mil...
47      The agreement was signed with Biohit Healthcar...
604     Kesko pursues a strategy of healthy , focused ...
3446    Vaisala , headquartered in Helsinki in Finland...
1166    Also , a six-year historic analysis is provide...
                              ...                        
4513    Dubai Nokia has announced the launch of `` Com...
2146    MADISON , Wis. , Feb. 6 - PRNewswire - -- Fisk...
1105    The report provides a comprehensive insight in...
1580    Pharmaceuticals - Netherlands This brand-new m...
3373    The technology will become available to busine...
Name: Text, Length: 3877, dtype: object

In [ ]:
import spacy
nlp=spacy.load("en_core_web_trf")

In [ ]:
nlp1 = spacy.blank("en")

db = DocBin()
for ind in train.index:
  text = train['Text'][ind]
  sentiment = train['Sentiment'][ind]

  doc = nlp(text)
  if (sentiment=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 0
  elif (sentiment=='negative'):
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
      doc.cats['neutral']  = 0
  else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 1
  doc.cats = sentiment
  db.add(doc)

db.to_disk("./train_sentiment") 

In [ ]:
db = DocBin()
for ind in test.index:
  text = test['Text'][ind]
  sentiment = test['Sentiment'][ind]

  doc = nlp(text)
  if (sentiment=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 0
  elif (sentiment=='negative'):
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
      doc.cats['neutral']  = 0
  else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 1
  doc.cats = sentiment
  db.add(doc)

db.to_disk("./test_sentiment") 